In [1]:
using IntervalConstraintProgramming, ValidatedNumerics

In [2]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

half_plane = @constraint x + y + z <= 1

Separator with variables x, y, z

In [37]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = setinverse(solid_torus ∩ half_plane, Y, 0.25);

  3.968720 seconds (17.33 M allocations: 463.621 MB, 11.76% gc time)


In [38]:
@time refine!(paving, 0.1)

 24.510120 seconds (117.16 M allocations: 2.985 GB, 15.25% gc time)


0.1

In [10]:
using JLD

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.4/JLD.ji for module JLD.
INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.4/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.4/Blosc.ji for module Blosc.


In [23]:
save("weird.jld", "boundary", paving.boundary)


In [25]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = setinverse(weird, Y, 0.5);

 53.493586 seconds (81.24 M allocations: 2.192 GB, 2.47% gc time)


In [3]:
pinch = @constraint x^2+y^2+z^2*(z-1)==0
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = setinverse(pinch, Y, 0.1);
inner = paving.boundary;

LoadError: LoadError: InterruptException:
while loading In[3], in expression starting on line 155

## 3D vis 

In [39]:
inner = paving.inner
boundary = paving.boundary;

In [11]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

In [40]:
positions = Point3{Float32}[Point3(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors1 = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.1f0)
        for i in 1:length(inner)]
colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
colors = vcat(colors1, colors2);

In [41]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors
println("Visualization packages loaded.")

cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

view(vis)


renderloop(window)

Visualization packages loaded.


Ptr{Void} @0x0000000000000000